In [ ]:
# ⚙️ Cài đặt thư viện cần thiết
!pip install -q torch torchvision pandas

In [ ]:
# 📁 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 🧠 Load mô hình và định nghĩa thông tin
import torch
from torchvision import models, transforms
import torch.nn.functional as F
from PIL import Image
import cv2, pickle
import pandas as pd
from pathlib import Path

video_path = "/content/drive/MyDrive/dispatch/video_shortened.mp4"
model_path = "/content/drive/MyDrive/dispatch/resnet18_dispatch.pt"
log_path = "/content/drive/MyDrive/dispatch/bbox_tracking_log.pkl"

output_csv = Path("classification_result.csv")
ui_objects_path = Path("ui_objects.pkl")

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = models.resnet18(pretrained=False, num_classes=6)
state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

# Transform ảnh đầu vào
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Class mapping
id_to_class = {
    0: 'dish_empty',
    1: 'dish_kakigori',
    2: 'dish_not_empty',
    3: 'tray_empty',
    4: 'tray_kakigori',
    5: 'tray_not_empty'
}

In [ ]:
# 🚀 Chạy phân loại
with open(log_path, "rb") as f:
    bbox_data = pickle.load(f)

cap = cv2.VideoCapture(video_path)
results = []
ui_data = {}

for frame_idx, objs in bbox_data.items():
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    success, frame = cap.read()
    if not success:
        print(f"[!] Không đọc được frame {frame_idx}")
        continue

    height, width = frame.shape[:2]
    ui_data[frame_idx] = []

    for obj in objs:
        cls = obj["object"]
        track_id = obj["id"]
        x, y, w, h = obj["bbox"]

        x = max(0, x)
        y = max(0, y)
        x2 = min(x + w, width)
        y2 = min(y + h, height)

        crop = frame[y:y2, x:x2]
        if crop.size == 0:
            continue

        crop_pil = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        input_tensor = transform(crop_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            prob = F.softmax(output, dim=1)
            pred = torch.argmax(prob, dim=1).item()
            confidence = prob[0][pred].item()
            label_name = id_to_class[pred]

        ui_data[frame_idx].append({
            "id": track_id,
            "object": cls,
            "status": label_name.split("_")[1],
            "bbox": obj["bbox"],
            "confidence": round(confidence, 4)
        })

        results.append({
            "frame": frame_idx,
            "track_id": track_id,
            "object": cls,
            "predicted": pred,
            "predicted_label": label_name,
            "confidence": round(confidence, 4)
        })

cap.release()

In [ ]:
# 💾 Lưu kết quả
pd.DataFrame(results).to_csv(output_csv, index=False)
with open(ui_objects_path, "wb") as f:
    pickle.dump(ui_data, f)

print("✅ Phân loại hoàn tất. Kết quả lưu tại:", output_csv, "và", ui_objects_path)

# 📤 Tải về
from google.colab import files
files.download("classification_result.csv")
files.download("ui_objects.pkl")